<a href="https://colab.research.google.com/github/MengOonLee/Python/blob/main/Plotly_Dash/Dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
pip install --no-cache-dir -qU \
  dash jupyter-dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 145.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 98.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 225.1 MB/s eta 0:00:00


In [2]:
from jupyter_dash import JupyterDash
import dash

app = JupyterDash(__name__)
app.layout = dash.html.Div(children=[
    dash.html.Div(style={
        'height':250, 'width':250, 'background-color':'red'
    }),
    dash.html.Div(children=[
        dash.html.H1("This box"),
        dash.html.H2("Another Title")],
        style={'background-color':'lightblue'}
    )
])
if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
ecom_sales = pd.read_csv('./Data/ecom_sales.csv')
ecom_sales['OrderValue'] = ecom_sales['Quantity'] * ecom_sales['UnitPrice']
ecom_sales['InvoiceDate'] = pd.to_datetime(ecom_sales['InvoiceDate'])\
    .dt.date
ecom_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
 8   OrderValue   541909 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 37.2+ MB


In [ ]:
ecom_sales

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,OrderValue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09,4.15,12680.0,France,16.60


In [ ]:
daily_sales = ecom_sales.groupby(by=['Country', 'InvoiceDate'])\
    ['OrderValue'].agg('sum')\
    .reset_index(name='Total Sales ($)')
daily_sales

,Country,InvoiceDate,Total Sales ($)
0,Australia,2010-12-01,358.25
1,Australia,2010-12-08,258.90
2,Australia,2010-12-14,-27.75
3,Australia,2010-12-17,415.70
4,Australia,2011-01-06,7154.38
...,...,...,...
1711,Unspecified,2011-07-28,546.43
1712,Unspecified,2011-08-22,531.03
1713,Unspecified,2011-09-02,286.85
1714,Unspecified,2011-11-16,333.20


In [ ]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
import dash



ecom_sales = pd.read_csv('./Data/ecom_sales.csv')
ecom_sales['OrderValue'] = ecom_sales['Quantity'] * ecom_sales['UnitPrice']
ecom_sales['InvoiceDate'] = pd.to_datetime(ecom_sales['InvoiceDate'])\
    .dt.date


country_sales = ecom_sales.groupby(by='Country')\
    ['OrderValue'].agg('sum')\
    .reset_index(name='Total Sales ($)')
bar_chart = px.bar(
    data_frame=country_sales,
    x='Total Sales ($)', y='Country',
    title='Total Sales by Country',
    orientation='h', color='Country'
)
bar_chart.update_layout({'bargap': 0.5})

app = JupyterDash(__name__)
app.layout = dash.html.Div(children=[
    dash.html.H1('Sales Proportion by Country'),
    dash.dcc.Graph(id='bar-chart', figure=bar_chart)
])

if __name__ == '__main__':
    app.run_server(host='0.0.0.0', debug=True)

Dash is running on http://0.0.0.0:8050/

Dash app running on http://0.0.0.0:8050/


## Dash components

In [ ]:
ecom_sales = pd.read_csv('./Data/ecom_sales.csv')
ecom_bar = ecom_sales.groupby(by='Country')\
    ['OrderValue'].agg('sum')\
    .reset_index(name='Total Sales ($)')
# Create the bar graph object
bar_fig = px.bar(
  # Set the DataFrame, x and y
  data_frame=ecom_bar, x='Total Sales ($)', y='Country',
  # Set the graph to be horizontal
  orientation='h', title='Total Sales by Country'
)
# Increase the gap between bars
bar_fig.update_layout({'bargap': 0.5})
# Create the Dash app
app = dash.Dash(__name__)
# Set up the layout with a single chart
app.layout = dash.dcc.Graph(
    id='bar-fig',
    # Insert the bar chart
    figure=bar_fig
)
# Set the app to run in development mode
if __name__ == '__main__':
    app.run_server(host='0.0.0.0')

In [ ]:
ecom_sales = pd.read_csv('./Data/ecom_sales.csv')

ecom_bar = ecom_sales.groupby(by='Country')\
    ['OrderValue'].agg('sum')\
    .reset_index(name='Total Sales ($)')

max_country = ecom_bar.sort_values(by='Total Sales ($)',
    ascending=False).loc[0]['Country']

bar_fig_country = px.bar(
    data_frame=ecom_bar, x='Total Sales ($)', y='Country',
    color='Country', color_discrete_map={
        'United Kingdom': 'lightblue',
        'Germany': 'orange',
        'France': 'darkblue',
        'Australia': 'green',
        'Hong Kong': 'red'}
)
app = dash.Dash(__name__)
app.layout = dash.html.Div(children=[
    dash.html.H1('Sales Proportion by Country'),
    # Add a div containing the bar figure
    dash.html.Div(dash.dcc.Graph(
        id='bar_graph', figure=bar_fig_country)
    ),
    # Add the H3
    dash.html.H3(f'The largest country by sales was {max_country}')    
])
if __name__ == '__main__':
    app.run_server(host='0.0.0.0')

In [ ]:
ecom_sales = pd.read_csv('./Data/ecom_sales.csv')
ecom_line = ecom_sales.groupby(['Year-Month','Country'])\
    ['OrderValue'].agg('sum')\
    .reset_index(name='Total Sales ($)')
ecom_bar = ecom_sales.groupby('Country')\
    ['OrderValue'].agg('sum')\
    .reset_index(name='Total Sales ($)')
line_graph = px.line(data_frame=ecom_line, x='Year-Month', y='Total Sales ($)',
    title='Total Sales by Month', color='Country')
bar_graph = px.bar(data_frame=ecom_bar, x='Total Sales ($)', y='Country',
    orientation='h',title='Total Sales by Country')

# Create the Dash app
app = dash.Dash()

# Set up the layout using an overall div
app.layout = html.Div(children=[
    # Add a H1
    html.H1('Sales by Country & Over Time'),
    # Add both graphs
    dcc.Graph(id='line_graph', figure=line_graph),
    dcc.Graph(id='bar_graph', figure=bar_graph)
])

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
ecom_sales = pd.read_csv('./Data/ecom_sales.csv')
ecom_line = ecom_sales.groupby('Year-Month')\
    ['OrderValue'].agg('sum')\
    .reset_index(name='Total Sales ($)')
line_fig = px.line(data_frame=ecom_line, x='Year-Month', y='Total Sales ($)',
    title='Total Sales by Month')
ecom_bar = ecom_sales.groupby('Country')\
    ['OrderValue'].agg('sum')\
    .reset_index(name='Total Sales ($)')
max_country = ecom_bar.sort_values(by='Total Sales ($)', ascending=False)\
    .loc[0]['Country']
bar_fig = px.bar(data_frame=ecom_bar, x='Total Sales ($)', y='Country',
    orientation='h', title='Total Sales by Country')

app = dash.Dash()

# Create the dash layout and overall div
app.layout = html.Div(children=[
    html.H1('Sales Figures'), 
    # Add a div containing the line figure
    html.Div(dcc.Graph(id='my-line-fig', figure=line_fig)), 
    # Add a div containing the bar figure
    html.Div(dcc.Graph(id='my-bar-fig', figure=bar_fig)), 
    # Add the H3
    html.H3(f'The largest country by sales was {max_country}')
])

if __name__ == '__main__':
    app.run_server(debug=True)

## HTML

In [ ]:
ecom_sales = pd.read_csv('./Data/ecom_sales.csv')
logo_link = './Data/e_com_logo.png'
ecom_bar = ecom_sales.groupby('Country')\
    ['OrderValue'].agg('sum')\
    .reset_index(name='Total Sales ($)')\
    .sort_values(by='Total Sales ($)', ascending=False)
top_country = ecom_bar.loc[0]['Country']            
bar_fig_country = px.bar(ecom_bar, x='Total Sales ($)', y='Country',
    color='Country', color_discrete_map={
        'United Kingdom':'lightblue', 'Germany':'orange', 'France':'darkblue',
        'Australia':'green', 'Hong Kong':'red'}
)         

app = dash.Dash()
app.layout = html.Div([
    # Add the company logo
    html.Img(src=logo_link),
    html.H1('Sales by Country'),
    html.Div(dcc.Graph(figure=bar_fig_country), 
        style={'width':'750px', 'margin':'auto'}),
    # Add an overall text-containing component
    html.Span(
        children=[
            # Add the top country text
            'This year, the most sales came from: ', 
            html.B(top_country),
            # Italicize copyright notice
            html.I(' Copyright E-Com INC')
        ])
    ],
    style={'text-align':'center', 'font-size':22}
)

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
ecom_sales = pd.read_csv('./Data/ecom_sales.csv')
logo_link = './Data/e_com_logo.png'
ecom_category = ecom_sales.groupby(['Major Category','Minor Category'])\
    .size().reset_index(name='Total Orders')\
    .sort_values(by='Total Orders', ascending=False)\
    .reset_index(drop=True)
num1_cat, num1_salesvol = ecom_category.loc[0].tolist()[1:3]
num2_cat, num2_salesvol = ecom_category.loc[1].tolist()[1:3]
ecom_bar = px.bar(ecom_category, x='Total Orders', y='Minor Category',
    color='Major Category')
ecom_bar.update_layout({'yaxis':{'dtick':1, 'categoryorder':'total ascending'}})             

app = dash.Dash()
app.layout = html.Div([
    html.Img(src=logo_link),
    html.H1('Top Sales Categories'),
    html.Div(dcc.Graph(figure=ecom_bar)),
    html.Span(
        children=[
            'The top 2 sales categories were:',
            # Set up an ordered list
            html.Ol(
                children=[
                    # Add two list elements with the top category variables
                    html.Li(children=[num1_cat, ' with ', num1_salesvol, ' sales volume']),
                    html.Li(children=[num2_cat, ' with ', num2_salesvol, ' sales volume'])
                ],
            style={'width':'350px', 'margin':'auto'}),
            # Add a line break before the copyright notice
            html.Br(),
            html.I('Copyright E-Com INC')
        ])
    ],
    style={'text-align':'center', 'font-size':22}
)

if __name__ == '__main__':
    app.run_server(debug=True)

## CSS

In [ ]:
ecom_sales = pd.read_csv('./Data/ecom_sales.csv')
logo_link = './Data/e_com_logo.png'
ecom_category = ecom_sales.groupby(['Major Category','Minor Category'])\
    .size().reset_index(name='Total Orders')\
    .sort_values(by='Total Orders', ascending=False)\
    .reset_index(drop=True)
top_cat = ecom_category.loc[0]['Minor Category']
ecom_bar = px.bar(ecom_category, x='Total Orders', y='Minor Category',
    color='Major Category')
ecom_bar.update_layout({'yaxis': {'dtick': 1, 
    'categoryorder': 'total ascending'},
    'paper_bgcolor': 'rgb(224, 255, 252)'})

app = dash.Dash()
app.layout = html.Div([
    html.Img(src=logo_link,
        # Set the size of the logo
        style={'width': '215px', 'height': '240px'}),
    html.H1('Top Sales Categories'),
    # Set the size of the bar graph
    html.Div(dcc.Graph(figure=ecom_bar, 
        style={'width': '500px', 'height': '350px', 'margin': 'auto'})),
    html.Br(),
    html.Span(children=[
        'The top category was: ',
        html.B(top_cat),
        html.Br(),
        html.I('Copyright E-Com INC',
            # Add a background color to the copyright notice
            style={'background-color': 'lightgrey'})
    ])
    # Add a background color to the entire app
    ], style={
        'text-align': 'center', 'font-size': 22, 
        'background-color':'rgb(224, 255, 252)'
    }
)

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
ecom_sales = pd.read_csv('./Data/ecom_sales.csv')
logo_link = './Data/e-comlogo_white.png'
ecom_category = ecom_sales.groupby(['Major Category','Minor Category'])\
    .size().reset_index(name='Total Orders')\
    .sort_values(by='Total Orders', ascending=False)\
    .reset_index(drop=True)
top_cat = ecom_category.loc[0]['Minor Category']
ecom_bar = px.bar(ecom_category, x='Total Orders', y='Minor Category',
    color='Major Category')

# Set the font color of the bar chart
ecom_bar.update_layout({'yaxis': {'dtick': 1,
    'categoryorder': 'total ascending'}, 'paper_bgcolor': 'black',
    'font': {'color': 'white'}})

app = dash.Dash()

app.layout = html.Div([
    # Set the new white-text image
    html.Img(src=logo_link,
        style={'width': '165px', 'height': '50px'}),
    html.H1('Top Sales Categories'),
    html.Div(dcc.Graph(figure=ecom_bar,
        style={'width': '500px', 'height': '350px', 'margin': 'auto'})),
    html.Br(),
    html.Span(children=[
        'The top category was: ',
        html.B(top_cat),
        html.Br(),
        html.I('Copyright E-Com INC')
    ])
    ], style={
        'text-align':'center', 'font-size':22,
        # Update the background color to the entire app
        'background-color': 'black',
        # Change the text color for the whole app
        'color': 'white'
    }
)

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
logo_link = './Data/e-comlogo.png'
ecom_sales = pd.read_csv('./Data/ecom_sales.csv')
ecom_line = ecom_sales.groupby('Year-Month')\
    ['OrderValue'].agg('sum')\
    .reset_index(name='TotalSales')
line_fig = px.line(data_frame=ecom_line, x='Year-Month', y='TotalSales',
    title='Total Sales by Month')
line_fig.update_layout({'paper_bgcolor': 'rgb(224, 255, 252)'}) 
ecom_bar = ecom_sales.groupby('Country')\
    ['OrderValue'].agg('sum')\
    .reset_index(name='TotalSales')
bar_fig = px.bar(data_frame=ecom_bar, x='TotalSales', y='Country',
    orientation='h', title='Total Sales by Country')
bar_fig.update_layout({'yaxis':{'dtick':1,
    'categoryorder':'total ascending'},
    'paper_bgcolor':'rgb(224, 255, 252)'}) 

app = dash.Dash()
app.layout = html.Div(children=[
    html.Div(children=[
        html.Img(src=logo_link, 
            # Place the logo side-by-side the H1 with required margin
            style={'display': 'inline-block', 'margin': '25px'}),
        html.H1(children=['Sales Figures'],
            # Make the H1 side-by-side with the logos
            style={'display': 'inline-block'}), 
        html.Img(src=logo_link,
            # Place the logo side-by-side the H1 with required margin
            style={'display': 'inline-block', 'margin': '25px'})
    ]),
    html.Div(
        dcc.Graph(figure=line_fig), 
        # Ensure graphs are correct size, side-by-side with required margin
        style={'width': '500px', 'display': 'inline-block',
            'margin': '5px'}), 
    html.Div(
        dcc.Graph(figure=bar_fig),
        # Ensure graphs are correct size, side-by-side with required margin
        style={'width': '350px', 'display': 'inline-block',
            'margin': '5px'}), 
    html.H3(f'The largest order quantity was {ecom_sales.Quantity.max()}')
    ], style={
        'text-align': 'center', 'font-size': 22, 
        'background-color': 'rgb(224, 255, 252)'
    }
)

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
ecom_sales = pd.read_csv('/Data/ecom_sales.csv')
logo_link = './Data/e-comlogo.png'
ecom_bar_major_cat = ecom_sales.groupby('Major Category')\
    ['OrderValue'].agg('sum')\
    .reset_index(name='Total Sales ($)')
ecom_bar_minor_cat = ecom_sales.groupby('Minor Category')\
    ['OrderValue'].agg('sum')\
    .reset_index(name='Total Sales ($)')
bar_fig_major_cat = px.bar(ecom_bar_major_cat, x='Total Sales ($)', y='Major Category',
    color='Major Category', color_discrete_map={'Clothes': 'blue', 'Kitchen': 'red',
        'Garden': 'green','Household': 'yellow'})
bar_fig_minor_cat = px.bar(ecom_bar_minor_cat, x='Total Sales ($)', y='Minor Category')                    

app = dash.Dash()
app.layout = html.Div([
    html.Img(src=logo_link,
        # Add margin to the logo
        style={'margin': '30px 0px 0px 0px'}),
    html.H1('Sales breakdowns'),
    html.Div(children=[
        dcc.Graph(
            # Style the graphs to appear side-by-side
            figure=bar_fig_major_cat,
            style={'display': 'inline-block'}),
          dcc.Graph(
            figure=bar_fig_minor_cat,
            style={'display': 'inline-block'}),
    ]),
    html.H2('Major Category',
        # Style the titles to appear side-by-side with a 2 pixel border
        style={'display': 'inline-block', 'border': '2px solid black',
            # Style the titles to have the correct spacings       
            'padding': '10px', 'margin': '10px 220px'}),
    html.H2('Minor Category',
        # Style the titles to appear side-by-side with a 2 pixel border
        style={'display': 'inline-block', 'border': '2px solid black',
            # Style the titles to have the correct spacings 
            'padding': '10px', 'margin': '10px 220px'}),
    ], style={
        'text-align': 'center', 'font-size': 22
    }
)

if __name__ == '__main__':
    app.run_server(debug=True)

## Callbacks and Components

In [ ]:
ecom_sales = pd.read_csv('./Data/ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'

app = dash.Dash(__name__)

app.layout = html.Div([
  html.Img(src=logo_link,style={'margin':'30px 0px 0px 0px' }),
  html.H1('Sales breakdowns'),
  html.Div(
    children=[
    html.Div(
        children=[
        html.H2('Controls'),
        html.Br(),
        html.H3('Country Select'),
        # Add a dropdown with identifier
        dcc.____(____='country_dd',
        # Set the available options with noted labels and values
        options=[
            {'____':'____', '____':'____'},
            {'____':'____', '____':'____'},
            {'____':'____', '____':'____'},
            {'____':'____', '____':'____'},
            {'____':'____', '____':'____'}],
            style={'width':'200px', 'margin':'0 auto'})
        ],
        style={'width':'350px', 'height':'350px', 'display':'inline-block', 'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
    html.Div(children=[
            # Add a graph component with identifier
            dcc.____(____='major_cat'),
            html.H2('Major Category', 
            style={ 'border':'2px solid black', 'width':'200px', 'margin':'0 auto'})
            ],
             style={'width':'700px','display':'inline-block'}
             ),
    ])], 
  style={'text-align':'center', 'display':'inline-block', 'width':'100%'}
  )

@app.callback(
    # Set the input and output of the callback to link the dropdown to the graph
    ____(component_id='____', component_property='____'),
    ____(component_id='____', component_property='____')
)

def update_plot(input_country):
    country_filter = 'All Countries'
    sales = ecom_sales.copy(deep=True)
    if input_country:
        country_filter = input_country
        sales = sales[sales['Country'] == country_filter]
    ecom_bar_major_cat = sales.groupby('Major Category')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)')
    bar_fig_major_cat = px.bar(
        title=f'Sales in {country_filter}', data_frame=ecom_bar_major_cat, x='Total Sales ($)', y='Major Category', color='Major Category',
                 color_discrete_map={'Clothes':'blue','Kitchen':'red','Garden':'green','Household':'yellow'})
    return bar_fig_major_cat


if __name__ == '__main__':
    app.run_server(debug=True)